In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

#Assign each file a variable name
file1 = 'GA_RevTranSessUser.csv'
file2 = 'GA_ChannelRevUsersTrans.csv'
file3 = 'GA_DeviceRevUsersTrans.csv'
file4 = 'DailyPlan.csv'
file5 = 'Media Spend.csv'

#Import each file
ga1_df = pd.read_csv(file1)
ga2_df = pd.read_csv(file2)
ga3_df = pd.read_csv(file3)
dailyplan_df = pd.read_csv(file4)
mediaspend_df = pd.read_csv(file5)

#Format the Date column for each file into the correct Date format
ga1_df['Date'] = pd.to_datetime(ga1_df['Date'], format='%Y%m%d')
ga2_df['Date'] = pd.to_datetime(ga2_df['Date'], format='%Y%m%d')
ga3_df['Date'] = pd.to_datetime(ga3_df['Date'], format='%Y%m%d')
dailyplan_df['Date'] = pd.to_datetime(dailyplan_df['Date'], format='%m/%d/%Y')
mediaspend_df['Date'] = pd.to_datetime(mediaspend_df['Date'], format='%m/%d/%Y')

#Check the date field format for each
#print(ga1_df.info())
#print(ga2_df.info())
#print(ga3_df.info())
#print(dailyplan_df.info())
print(mediaspend_df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4295 entries, 0 to 4294
Data columns (total 3 columns):
Date       4295 non-null datetime64[ns]
Channel    4295 non-null object
Spend      4295 non-null float64
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 100.7+ KB
None


In [2]:
#ga2 included rows for both "Affiliates" and "Affiliate". Update occurances of "Affiliates" to "Affiliate"
ga2_df = ga2_df.replace({'Affiliates':'Affiliate'})

#ga2 now has multiple rows for the same date for Affiliate. Sum Users, Revenue & Transactions when two rows occur
ga2_df = ga2_df.groupby(['Date','Default Channel Grouping'])['Users', 'Revenue', 'Transactions'].sum().reset_index()

#Pivot the ga2, ga3 & media spend files so that there is a single row for each date
ga2_df = ga2_df.pivot(index='Date', columns=('Default Channel Grouping'))
ga3_df = ga3_df.pivot(index='Date', columns=('Device Category'))
mediaspend_df = mediaspend_df.pivot(index='Date', columns=('Channel'))
#print(ga2_df.info())
#print(ga3_df.info())
print(mediaspend_df.head())


               Spend                                              
Channel    Affiliate     CSE  Display  PLA Paid Search Paid Social
Date                                                              
2016-01-01   9093.52  310.09  1222.32  0.0    24148.87     1871.12
2016-01-02   7844.25  522.61  1312.48  0.0    26192.79     1908.61
2016-01-03   7131.42  494.37  1418.63  0.0    26876.02     1931.49
2016-01-04   9670.25  371.78  1641.33  0.0    18381.37     1865.61
2016-01-05   5719.90  388.59  1942.57  0.0    16203.04     1514.85


In [3]:
print(dailyplan_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 9 columns):
Date                              420 non-null datetime64[ns]
TY Event                          283 non-null object
TY Email                          274 non-null object
Daily Revenue Demand Plan         420 non-null int64
Daily Traffic Plan                420 non-null int64
Daily Net Shipped Revenue Plan    420 non-null int64
Orders Plan                       420 non-null float64
Units Plan                        420 non-null float64
Daily Margin $ Plan               420 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 29.6+ KB
None


In [4]:
ga2_df = ga2_df.fillna(0)
ga2_df.info()
mediaspend_df = mediaspend_df.fillna(0)
mediaspend_df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 394 entries, 2016-12-04 to 2018-01-01
Data columns (total 39 columns):
(Users, (Other))                  394 non-null float64
(Users, Affiliate)                394 non-null float64
(Users, CSE)                      394 non-null float64
(Users, Direct)                   394 non-null float64
(Users, Display)                  394 non-null float64
(Users, Email)                    394 non-null float64
(Users, Organic Search)           394 non-null float64
(Users, Organic Social)           394 non-null float64
(Users, Paid Search)              394 non-null float64
(Users, Paid Social)              394 non-null float64
(Users, Partnerships)             394 non-null float64
(Users, Referral)                 394 non-null float64
(Users, Social)                   394 non-null float64
(Revenue, (Other))                394 non-null float64
(Revenue, Affiliate)              394 non-null float64
(Revenue, CSE)                    394 non-null floa

In [5]:
#Create a new column with either 1 or 0, 1 if there was an email that day, 0 if no email
dailyplan_df['Email Day'] = dailyplan_df['TY Email'].notnull().mul(1)
print(dailyplan_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 10 columns):
Date                              420 non-null datetime64[ns]
TY Event                          283 non-null object
TY Email                          274 non-null object
Daily Revenue Demand Plan         420 non-null int64
Daily Traffic Plan                420 non-null int64
Daily Net Shipped Revenue Plan    420 non-null int64
Orders Plan                       420 non-null float64
Units Plan                        420 non-null float64
Daily Margin $ Plan               420 non-null int64
Email Day                         420 non-null int32
dtypes: datetime64[ns](1), float64(2), int32(1), int64(4), object(2)
memory usage: 31.2+ KB
None


In [6]:
#Create a new column for promo type and extract the type of sale from TY Event
dailyplan_df['Promo Type'] = dailyplan_df['TY Event'].str.extract('(sale on sale|Sale on Sale|SOS|sos|markdowns|Markdowns|SS|Surprise Sale|surprise sale|ss|GWP|DOTD|DOD|New Arrivals|new arrivals|Friends & Family|friends & family|SWP|Employee Appreciation|employee appreciation)')

#Create a new column for each promo type, put 1 if occured 0 if not
dailyplan_df['SS'] = [1 if ele == 'surprise sale' or ele == 'ss' or ele == 'Surprise Sale' or ele == 'SS' else 0 for ele in dailyplan_df['Promo Type']]
dailyplan_df['MD'] = [1 if ele == 'Markdowns' or ele == 'markdowns' else 0 for ele in dailyplan_df['Promo Type']]
dailyplan_df['GWP'] = [1 if ele == 'GWP' else 0 for ele in dailyplan_df['Promo Type']]
dailyplan_df['FF'] = [1 if ele == 'Friends & Family' or ele == 'friends & family' else 0 for ele in dailyplan_df['Promo Type']]
dailyplan_df['DOTD'] = [1 if ele == 'DOD' or ele == 'DOTD' else 0 for ele in dailyplan_df['Promo Type']]
dailyplan_df['NA'] = [1 if ele == 'New Arrivals' or ele == 'new arrivals' else 0 for ele in dailyplan_df['Promo Type']]
dailyplan_df['SOS'] = [1 if ele == 'Sale on Sale' or ele == 'sale on sale'or ele == 'SOS' or ele == 'sos' else 0 for ele in dailyplan_df['Promo Type']]
dailyplan_df['SWP'] = [1 if ele == 'SWP' else 0 for ele in dailyplan_df['Promo Type']]
dailyplan_df['EA'] = [1 if ele == 'Employee Appreciation' or ele == 'employee appreciation' else 0 for ele in dailyplan_df['Promo Type']]
dailyplan_df['Sale Day'] = [1 if ele == 'surprise sale' or ele == 'ss' or ele == 'Surprise Sale' or ele == 'SS' or ele == 'Markdowns' or ele == 'markdowns' or ele == 'GWP' or ele == 'Friends & Family' or ele == 'friends & family' or ele == 'DOD' or ele == 'DOTD' or ele == 'Sale on Sale' or ele == 'sale on sale'or ele == 'SOS' or ele == 'sos' or ele == 'SWP' else 0 for ele in dailyplan_df['Promo Type']]

#Create calculated fields for AOV and CVR
dailyplan_df['AOV Plan'] = dailyplan_df['Daily Revenue Demand Plan']/dailyplan_df['Orders Plan']
dailyplan_df['CVR Plan'] = dailyplan_df['Orders Plan']/dailyplan_df['Daily Traffic Plan'] * 100

#Drop columns no longer necessary
dailyplan_df = dailyplan_df.drop(['TY Event', 'TY Email', 'Promo Type'], axis=1)

print(dailyplan_df.head())

        Date  Daily Revenue Demand Plan  Daily Traffic Plan  \
0 2016-12-04                    1830000              465923   
1 2016-12-05                     815591              275747   
2 2016-12-06                     744743              383480   
3 2016-12-07                     936497              329111   
4 2016-12-08                    1115910              373931   

   Daily Net Shipped Revenue Plan  Orders Plan  Units Plan  \
0                          944925      12607.0     17069.0   
1                         1239086       5972.0      8925.0   
2                          561874       5343.0      7488.0   
3                          467726       6880.0      9563.0   
4                          835736       8372.0     12144.0   

   Daily Margin $ Plan  Email Day  SS  MD  GWP  FF  DOTD  NA  SOS  SWP  EA  \
0               635025          1   0   0    0   0     1   0    0    0   0   
1               832713          0   0   0    0   0     0   0    0    0   1   
2             

C:\Users\jrenaud\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


In [7]:
#Converting Avg. Session Duration to total Seconds
ga1_df['Avg. Session Duration'].astype('str')
ga_new_df = pd.DataFrame(ga1_df['Avg. Session Duration'].str.split(':').tolist(), columns='hours minutes seconds'.split()).astype('float')
ga_new_df['Hours to Seconds'] = ga_new_df['hours'] * 3600
ga_new_df['Min to Seconds'] = ga_new_df['minutes'] * 60
ga_new_df['Avg. Session Duration Seconds'] = ga_new_df['Hours to Seconds'] + ga_new_df['Min to Seconds'] + ga_new_df['seconds']
ga_new_df = ga_new_df.drop(['hours', 'minutes', 'seconds', 'Hours to Seconds', 'Min to Seconds'], axis=1)
print ga_new_df.head()

   Avg. Session Duration Seconds
0                          212.0
1                          203.0
2                          170.0
3                          198.0
4                          202.0


In [8]:
#Concatenate the new column to the original dataset
ga1_df = pd.concat([ga1_df, ga_new_df], axis = 1)
#print ga1_df.head()

ga1_df = ga1_df.drop(['Avg. Session Duration'], axis=1)
print ga1_df.head()


        Date    Revenue     Users  Sessions  Transactions  Quantity   Bounces  \
0 2015-12-06  573167.56  213409.0  268964.0        4023.0    7665.0  106838.0   
1 2015-12-07  785958.62  245784.0  302680.0        5556.0    7811.0   98690.0   
2 2015-12-08  847696.81  361013.0  439461.0        5747.0    6472.0  164418.0   
3 2015-12-09  522765.52  268389.0  334194.0        4693.0    7309.0  107965.0   
4 2015-12-10  854602.90  296167.0  370017.0        7184.0    8943.0  122911.0   

   Pages / Session Session Duration  Avg. Session Duration Seconds  
0             4.56      18585:05:00                          212.0  
1             4.39      16441:15:01                          203.0  
2             3.73      18893:46:36                          170.0  
3             4.31      16892:43:11                          198.0  
4             4.30      18903:54:45                          202.0  


In [9]:
#Shift GA data forward 1 and 2 days
ga1_df['Revenue Yesterday'] = ga1_df['Revenue'].shift(1)
ga1_df['Revenue Two Days Ago'] = ga1_df['Revenue'].shift(2)

ga1_df['Users Yesterday'] = ga1_df['Users'].shift(1)
ga1_df['Users Two Days Ago'] = ga1_df['Users'].shift(2)

ga1_df['Sessions Yesterday'] = ga1_df['Sessions'].shift(1)
ga1_df['Sessions Two Days Ago'] = ga1_df['Sessions'].shift(2)

ga1_df['Transactions Yesterday'] = ga1_df['Transactions'].shift(1)
ga1_df['Transactions Two Days Ago'] = ga1_df['Transactions'].shift(2)

ga1_df['Quantity Yesterday'] = ga1_df['Quantity'].shift(1)
ga1_df['Quantity Two Days Ago'] = ga1_df['Quantity'].shift(2)

ga1_df['Bounces Yesterday'] = ga1_df['Bounces'].shift(1)
ga1_df['Bounces Two Days Ago'] = ga1_df['Bounces'].shift(2)

ga1_df['Pages / Sess Yesterday'] = ga1_df['Pages / Session'].shift(1)
ga1_df['Pages / Sess Two Days Ago'] = ga1_df['Pages / Session'].shift(2)

ga1_df['Avg. Sess Dur Sec Yesterday'] = ga1_df['Avg. Session Duration Seconds'].shift(1)
ga1_df['Avg. Sess Dur Sec Two Days Ago'] = ga1_df['Avg. Session Duration Seconds'].shift(2)

ga1_df = ga1_df.fillna(0)
print(ga1_df.head())

        Date    Revenue     Users  Sessions  Transactions  Quantity   Bounces  \
0 2015-12-06  573167.56  213409.0  268964.0        4023.0    7665.0  106838.0   
1 2015-12-07  785958.62  245784.0  302680.0        5556.0    7811.0   98690.0   
2 2015-12-08  847696.81  361013.0  439461.0        5747.0    6472.0  164418.0   
3 2015-12-09  522765.52  268389.0  334194.0        4693.0    7309.0  107965.0   
4 2015-12-10  854602.90  296167.0  370017.0        7184.0    8943.0  122911.0   

   Pages / Session Session Duration  Avg. Session Duration Seconds  \
0             4.56      18585:05:00                          212.0   
1             4.39      16441:15:01                          203.0   
2             3.73      18893:46:36                          170.0   
3             4.31      16892:43:11                          198.0   
4             4.30      18903:54:45                          202.0   

                ...                Transactions Yesterday  \
0               ...            

In [10]:
#Shift GA data forward 364 days (1 fiscal year later date equivalent)
ga1_df['Revenue LY'] = ga1_df['Revenue'].shift(364)

ga1_df['Users LY'] = ga1_df['Users'].shift(364)

ga1_df['Sessions LY'] = ga1_df['Sessions'].shift(364)

ga1_df['Transactions LY'] = ga1_df['Transactions'].shift(364)

ga1_df['Quantity LY'] = ga1_df['Quantity'].shift(364)

ga1_df['Bounces LY'] = ga1_df['Bounces'].shift(364)

ga1_df['Pages / Sess LY'] = ga1_df['Pages / Session'].shift(364)

ga1_df['Avg. Sess LY'] = ga1_df['Avg. Session Duration Seconds'].shift(364)

ga1_df = ga1_df.fillna(0)
print(ga1_df.tail())

          Date     Revenue     Users  Sessions  Transactions  Quantity  \
753 2017-12-28  1034391.22  368503.0  459265.0        8846.0   13924.0   
754 2017-12-29   877046.00  317092.0  402293.0        7278.0   11194.0   
755 2017-12-30   633126.51  226532.0  282580.0        5195.0    7756.0   
756 2017-12-31   526626.20  186563.0  228366.0        4324.0    6477.0   
757 2018-01-01   586981.69  243234.0  299631.0        5031.0    7397.0   

      Bounces  Pages / Session Session Duration  \
753  169836.0             5.12      33878:49:49   
754  158562.0             4.89      27118:27:40   
755  107280.0             5.09      19611:50:41   
756   88784.0             5.02      15439:02:07   
757  120791.0             4.72      18871:17:22   

     Avg. Session Duration Seconds      ...       Avg. Sess Dur Sec Yesterday  \
753                          266.0      ...                             282.0   
754                          243.0      ...                             266.0   
755  

In [11]:
#Shift GA Device Data forward 1 and 2 days

ga3_df['Desktop Rev Yesterday'] = ga3_df[ga3_df.columns[3]].shift(1)
ga3_df['Desktop Rev Two Days Ago'] = ga3_df[ga3_df.columns[3]].shift(2)

ga3_df['Mobile Rev Yesterday'] = ga3_df[ga3_df.columns[4]].shift(1)
ga3_df['Mobile Rev Two Days Ago'] = ga3_df[ga3_df.columns[4]].shift(2)

ga3_df['Tablet Rev Yesterday'] = ga3_df[ga3_df.columns[5]].shift(1)
ga3_df['Tablet Rev Two Days Ago'] = ga3_df[ga3_df.columns[5]].shift(2)

ga3_df['Desktop Users Yesterday'] = ga3_df[ga3_df.columns[0]].shift(1)
ga3_df['Desktop Users Two Days Ago'] = ga3_df[ga3_df.columns[0]].shift(2)

ga3_df['Mobile Users Yesterday'] = ga3_df[ga3_df.columns[1]].shift(1)
ga3_df['Mobile Users Two Days Ago'] = ga3_df[ga3_df.columns[1]].shift(2)

ga3_df['Tablet Users Yesterday'] = ga3_df[ga3_df.columns[2]].shift(1)
ga3_df['Tablet Users Two Days Ago'] = ga3_df[ga3_df.columns[2]].shift(2)

ga3_df['Desktop Trans Yesterday'] = ga3_df[ga3_df.columns[6]].shift(1)
ga3_df['Desktop Trans Two Days Ago'] = ga3_df[ga3_df.columns[6]].shift(2)

ga3_df['Mobile Trans Yesterday'] = ga3_df[ga3_df.columns[7]].shift(1)
ga3_df['Mobile Trans Two Days Ago'] = ga3_df[ga3_df.columns[7]].shift(2)

ga3_df['Tablet Trans Yesterday'] = ga3_df[ga3_df.columns[8]].shift(1)
ga3_df['Tablet Trans Two Days Ago'] = ga3_df[ga3_df.columns[8]].shift(2)

ga3_df = ga3_df.fillna(0)

print(ga3_df.head())

                    Users                       Revenue                       \
Device Category   desktop    mobile   tablet    desktop     mobile    tablet   
Date                                                                           
2016-12-04        83675.0  189372.0  31895.0  590031.95  496103.50  135234.8   
2016-12-05        95250.0  116595.0  19209.0  443200.95  198139.15   62934.0   
2016-12-06       109817.0  151259.0  22420.0  411751.27  158087.81   49458.4   
2016-12-07       104945.0  136524.0  21465.0  412166.56  173308.09   50660.7   
2016-12-08       120379.0  170121.0  36949.0  492278.15  229536.95   55609.2   

                Transactions                Desktop Rev Yesterday  \
Device Category      desktop  mobile tablet                         
Date                                                                
2016-12-04            3741.0  3107.0  819.0                  0.00   
2016-12-05            3169.0  1416.0  440.0             590031.95   
2016-12-06    

In [12]:
#Shift Channel Data 1 day forward

#Revenue
ga2_df['Other Rev Yesterday'] = ga2_df[ga2_df.columns[13]].shift(1)
ga2_df['Affiliate Rev Yesterday'] = ga2_df[ga2_df.columns[14]].shift(1)
ga2_df['CSE Rev Yesterday'] = ga2_df[ga2_df.columns[15]].shift(1)
ga2_df['Direct Rev Yesterday'] = ga2_df[ga2_df.columns[16]].shift(1)
ga2_df['Display Rev Yesterday'] = ga2_df[ga2_df.columns[17]].shift(1)
ga2_df['Email Rev Yesterday'] = ga2_df[ga2_df.columns[18]].shift(1)
ga2_df['Org Search Rev Yesterday'] = ga2_df[ga2_df.columns[19]].shift(1)
ga2_df['Org Soc Rev Yesterday'] = ga2_df[ga2_df.columns[20]].shift(1)
ga2_df['Paid Search Rev Yesterday'] = ga2_df[ga2_df.columns[21]].shift(1)
ga2_df['Paid Soc Rev Yesterday'] = ga2_df[ga2_df.columns[22]].shift(1)
ga2_df['Partnership Rev Yesterday'] = ga2_df[ga2_df.columns[23]].shift(1)
ga2_df['Referral Rev Yesterday'] = ga2_df[ga2_df.columns[24]].shift(1)

#Users
ga2_df['Other Users Yesterday'] = ga2_df[ga2_df.columns[0]].shift(1)
ga2_df['Affiliate Users Yesterday'] = ga2_df[ga2_df.columns[1]].shift(1)
ga2_df['CSE Users Yesterday'] = ga2_df[ga2_df.columns[2]].shift(1)
ga2_df['Direct Users Yesterday'] = ga2_df[ga2_df.columns[3]].shift(1)
ga2_df['Display Users Yesterday'] = ga2_df[ga2_df.columns[4]].shift(1)
ga2_df['Email Users Yesterday'] = ga2_df[ga2_df.columns[5]].shift(1)
ga2_df['Org Search Users Yesterday'] = ga2_df[ga2_df.columns[6]].shift(1)
ga2_df['Org Soc Users Yesterday'] = ga2_df[ga2_df.columns[7]].shift(1)
ga2_df['Paid Search Users Yesterday'] = ga2_df[ga2_df.columns[8]].shift(1)
ga2_df['Paid Soc Users Yesterday'] = ga2_df[ga2_df.columns[9]].shift(1)
ga2_df['Partnership Users Yesterday'] = ga2_df[ga2_df.columns[10]].shift(1)
ga2_df['Referral Users Yesterday'] = ga2_df[ga2_df.columns[11]].shift(1)

#Transactions
ga2_df['Other Transactions Yesterday'] = ga2_df[ga2_df.columns[25]].shift(1)
ga2_df['Affiliate Transactions Yesterday'] = ga2_df[ga2_df.columns[26]].shift(1)
ga2_df['CSE Transactions Yesterday'] = ga2_df[ga2_df.columns[27]].shift(1)
ga2_df['Direct Transactions Yesterday'] = ga2_df[ga2_df.columns[28]].shift(1)
ga2_df['Display Transactions Yesterday'] = ga2_df[ga2_df.columns[29]].shift(1)
ga2_df['Email Transactions Yesterday'] = ga2_df[ga2_df.columns[30]].shift(1)
ga2_df['Org Search Transactions Yesterday'] = ga2_df[ga2_df.columns[31]].shift(1)
ga2_df['Org Soc Transactions Yesterday'] = ga2_df[ga2_df.columns[32]].shift(1)
ga2_df['Paid Search Transactions Yesterday'] = ga2_df[ga2_df.columns[33]].shift(1)
ga2_df['Paid Soc Transactions Yesterday'] = ga2_df[ga2_df.columns[34]].shift(1)
ga2_df['Partnership Transactions Yesterday'] = ga2_df[ga2_df.columns[35]].shift(1)
ga2_df['Referral Transactions Yesterday'] = ga2_df[ga2_df.columns[36]].shift(1)

ga2_df = ga2_df.fillna(0)
print(ga2_df.head())

                           Users                                               \
Default Channel Grouping (Other) Affiliate     CSE   Direct  Display    Email   
Date                                                                            
2016-12-04                  71.0   11819.0  1277.0  33488.0   6123.0  91991.0   
2016-12-05                 704.0   10391.0  1376.0  31868.0   4457.0  36808.0   
2016-12-06                 444.0   10372.0   891.0  30780.0   4461.0  98907.0   
2016-12-07                 144.0   23522.0  1132.0  30330.0   5299.0  55409.0   
2016-12-08                  77.0   18435.0  1635.0  31412.0  26715.0  99078.0   

                                                                    \
Default Channel Grouping Organic Search Organic Social Paid Search   
Date                                                                 
2016-12-04                      78178.0         2557.0     62804.0   
2016-12-05                      71858.0         3150.0     59878.0   
2

In [13]:
#Shift Media Spend data forward 1, 2 and 364 days
print(mediaspend_df.head())

mediaspend_df['Spend Aff Yesterday'] = mediaspend_df[mediaspend_df.columns[0]].shift(1)
mediaspend_df['Spend Aff Two Days Ago'] = mediaspend_df[mediaspend_df.columns[0]].shift(2)
mediaspend_df['Spend Aff LY'] = mediaspend_df[mediaspend_df.columns[0]].shift(364)

mediaspend_df['Spend CSE Yesterday'] = mediaspend_df[mediaspend_df.columns[1]].shift(1)
mediaspend_df['Spend CSE Two Days Ago'] = mediaspend_df[mediaspend_df.columns[1]].shift(2)
mediaspend_df['Spend CSE LY'] = mediaspend_df[mediaspend_df.columns[1]].shift(364)

mediaspend_df['Spend Display Yesterday'] = mediaspend_df[mediaspend_df.columns[2]].shift(1)
mediaspend_df['Spend Display Two Days Ago'] = mediaspend_df[mediaspend_df.columns[2]].shift(2)
mediaspend_df['Spend Display LY'] = mediaspend_df[mediaspend_df.columns[2]].shift(364)

mediaspend_df['Spend PLA Yesterday'] = mediaspend_df[mediaspend_df.columns[3]].shift(1)
mediaspend_df['Spend PLA Two Days Ago'] = mediaspend_df[mediaspend_df.columns[3]].shift(2)
mediaspend_df['Spend PLA LY'] = mediaspend_df[mediaspend_df.columns[3]].shift(364)

mediaspend_df['Spend PSearch Yesterday'] = mediaspend_df[mediaspend_df.columns[4]].shift(1)
mediaspend_df['Spend PSearch Two Days Ago'] = mediaspend_df[mediaspend_df.columns[4]].shift(2)
mediaspend_df['Spend PSearch LY'] = mediaspend_df[mediaspend_df.columns[4]].shift(364)

mediaspend_df['Spend PSocial Yesterday'] = mediaspend_df[mediaspend_df.columns[5]].shift(1)
mediaspend_df['Spend PSocial Two Days Ago'] = mediaspend_df[mediaspend_df.columns[5]].shift(2)
mediaspend_df['Spend PSocial LY'] = mediaspend_df[mediaspend_df.columns[5]].shift(364)

mediaspend_df = mediaspend_df.fillna(0)
print(mediaspend_df.head())

               Spend                                              
Channel    Affiliate     CSE  Display  PLA Paid Search Paid Social
Date                                                              
2016-01-01   9093.52  310.09  1222.32  0.0    24148.87     1871.12
2016-01-02   7844.25  522.61  1312.48  0.0    26192.79     1908.61
2016-01-03   7131.42  494.37  1418.63  0.0    26876.02     1931.49
2016-01-04   9670.25  371.78  1641.33  0.0    18381.37     1865.61
2016-01-05   5719.90  388.59  1942.57  0.0    16203.04     1514.85
               Spend                                                \
Channel    Affiliate     CSE  Display  PLA Paid Search Paid Social   
Date                                                                 
2016-01-01   9093.52  310.09  1222.32  0.0    24148.87     1871.12   
2016-01-02   7844.25  522.61  1312.48  0.0    26192.79     1908.61   
2016-01-03   7131.42  494.37  1418.63  0.0    26876.02     1931.49   
2016-01-04   9670.25  371.78  1641.33  0.0  

In [14]:
#Create 1 DataFrame for all 5 files
#ga1_df = ga1_df.drop(['Avg. Session Duration Seconds.1'], axis=1)
ga1_ga2_joined_df = ga1_df.join(ga2_df, how='inner', on='Date')
ga_all_joined_df = ga1_ga2_joined_df.join(ga3_df, how='inner', on='Date')
plan_spend_df = dailyplan_df.join(mediaspend_df, how='inner', on='Date')
all_df = ga_all_joined_df.merge(plan_spend_df, how='inner', on='Date')
print(all_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 393 entries, 0 to 392
Columns: 179 entries, Date to (Spend PSocial LY, )
dtypes: datetime64[ns](1), float64(162), int32(1), int64(14), object(1)
memory usage: 551.1+ KB
None


C:\Users\jrenaud\AppData\Local\Continuum\Anaconda2\lib\site-packages\pandas\core\reshape\merge.py:551: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [15]:
#Create a new column with 1 if they reached or exceeded goal and 0 if they did not
all_df['Rev Goal Reached'] = np.where(all_df['Revenue'] >= all_df['Daily Revenue Demand Plan'], 1, 0)
all_df['Users Goal Reached'] = np.where(all_df['Users'] >= all_df['Daily Traffic Plan'], 1, 0)
all_df['Orders Goal Reached'] = np.where(all_df['Transactions'] >= all_df['Orders Plan'], 1, 0)
all_df['Units Goal Reached'] = np.where(all_df['Quantity'] >= all_df['Units Plan'], 1, 0)
all_df['AOV Goal Reached'] = np.where(all_df['Revenue']/all_df['Transactions'] >= all_df['AOV Plan'], 1, 0)
all_df['CVR Goal Reached'] = np.where(all_df['Transactions']/all_df['Users'] * 100 >= all_df['CVR Plan'], 1, 0)

#Create a new column to say which goals were and weren't reached yesterday
all_df['Rev Goal Reached Yesterday'] = all_df['Rev Goal Reached'].shift(1)
all_df['Users Goal Reached Yesterday'] = all_df['Rev Goal Reached'].shift(1)
all_df['Orders Goal Reached Yesterday'] = all_df['Rev Goal Reached'].shift(1)
all_df['Units Goal Reached Yesterday'] = all_df['Rev Goal Reached'].shift(1)
all_df['AOV Goal Reached Yesterday'] = all_df['Rev Goal Reached'].shift(1)
all_df['CVR Goal Reached Yesterday'] = all_df['Rev Goal Reached'].shift(1)

all_df = all_df.fillna(0)

all_df = all_df.drop(['Users Goal Reached', 'Orders Goal Reached', 'Units Goal Reached', 'AOV Goal Reached', 'CVR Goal Reached'], axis=1)


print(all_df['Rev Goal Reached Yesterday'].tail())

388    1.0
389    1.0
390    1.0
391    1.0
392    0.0
Name: Rev Goal Reached Yesterday, dtype: float64


In [16]:
all_df.to_csv('C:\Users\jrenaud\Documents\Springboard\Project 1\Data Outputs\\all_df.csv')

In [18]:
all_df.to_csv('all_df.csv')
#ga2_df.to_csv('ga2_df.csv')

In [19]:
all_df.head()

,Date,Revenue,Users,Sessions,Transactions,Quantity,Bounces,Pages / Session,Session Duration,Avg. Session Duration Seconds,...,"(Spend PSocial Yesterday, )","(Spend PSocial Two Days Ago, )","(Spend PSocial LY, )",Rev Goal Reached,Rev Goal Reached Yesterday,Users Goal Reached Yesterday,Orders Goal Reached Yesterday,Units Goal Reached Yesterday,AOV Goal Reached Yesterday,CVR Goal Reached Yesterday
0,2016-12-04,1221370.25,306549.0,379353.0,7667.0,10094.0,119198.0,4.36,22102:57:44,210.0,...,2893.22,3194.77,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-12-05,704274.10,233431.0,281685.0,5025.0,7252.0,95546.0,4.24,16100:14:14,206.0,...,10783.70,2893.22,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016-12-06,619297.48,282736.0,345140.0,4597.0,6169.0,139158.0,3.72,17118:51:58,179.0,...,3449.71,10783.70,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-12-07,636135.35,259674.0,321574.0,5004.0,6776.0,121949.0,4.01,17390:56:33,195.0,...,3945.31,3449.71,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016-12-08,777424.30,324932.0,404220.0,6542.0,9046.0,173149.0,3.82,20827:29:21,185.0,...,4430.08,3945.31,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
